**TODO:**

- 3-5 CNN models
- From simple 1 layer fully connected to transfer learning
- May review ensemble
- Need to review books vs movies(tvshows) model

In [ ]:
import keras
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from matplotlib import pyplot as plt
import time
import helper_keras
import numpy as np
import pandas as pd
import os

#### Data Paths

In [ ]:
notbook_path = '/home/jason/DeepLearning/github/goodscifi/development/notebooks/'
home_path = '/home/jason/DeepLearning/github/goodscifi/development/'
data_path = '/home/jason/DeepLearning/github/goodscifi/development/data/'
model_path = '/home/jason/DeepLearning/github/goodscifi/development/models/'

In [ ]:
# Not using 'sample' since I have less than 6k total data
training_data = data_path+'dataset/books/train/'
validation_data = data_path+'dataset/books/valid/'

#### Input Data

In [ ]:
train_batches = helper_keras.get_batches(training_data) # update target_size
val_batches = helper_keras.get_batches(validation_data)

In [ ]:
# Create numpy array of data
train_data_array = helper_keras.get_data(training_data)
val_data_array = helper_keras.get_data(validation_data)

In [ ]:
# Save and use for datagen.fit
helper_keras.save_array(data_path+'dataset/books/train_data.bcolz',train_data_array)
helper_keras.save_array(data_path+'dataset/books/val_data.bcolz',val_data_array)

#### Load Data Array

In [ ]:
train_data_array = helper_keras.load_array(data_path+'dataset/books/train_data.bcolz')
val_data_array = helper_keras.load_array(data_path+'dataset/books/val_data.bcolz')

#### Data Augmentation

In [ ]:
data_gen_args = dict(featurewise_center=True,
                     featurewise_std_normalization=True)

image_datagen = ImageDataGenerator(**data_gen_args)

In [ ]:
# Provide the same seed and keyword arguments to the fit and flow methods
seed = 1
image_datagen.fit(train_data_array, augment=True, seed=seed)

In [ ]:
# Use this to standardize images for prediction
image_datagen.standardize?

In [ ]:
# Get train_batches with datagen augmentation (normalized)
# update target_size and batch_size

# update 'get_batches' with seed
train_batches_aug = helper_keras.get_batches(training_data,
                                           gen=image_datagen, 
                                           batch_size=8,
                                           target_size=(150,150)) 
val_batches_aug = helper_keras.get_batches(validation_data,
                                         gen=image_datagen, 
                                         batch_size=8,
                                         target_size=(150,150))

In [ ]:
# Set input_shape for all models
input_shape = train_batches_aug.image_shape; input_shape

In [ ]:
# Set steps per epoch for train & validation data
steps_per_epoch = train_batches_aug.samples/train_batches_aug.batch_size
val_steps = val_batches_aug.samples/val_batches_aug.batch_size

print('steps per epoch:', steps_per_epoch)
print('validation steps:', val_steps)

#### Fully Connected NN

In [ ]:
import nn_dense
model = nn_dense.model(input_shape)
model.summary()

#### Simple CNN

In [ ]:
import nn_simple
model = nn_simple.model(input_shape)
model.summary()

#### Deep CNN

In [ ]:
import nn_deep
model = nn_deep.model(input_shape)
model.summary()

In [ ]:
import nn_deep_sep_conv
model = nn_deep_sep_conv.model(input_shape)
model.summary()

In [ ]:
import nn_mobilenet
# model = nn_mobilenet.model(input_shape=input_shape)
# model.summary()

In [ ]:
# Test with movies and books
# VGG16 pre-trained -> https://github.com/jph00/part2/blob/master/vgg16_avg.py
# -- 1. without top and new conv or dense layer for classes
# -- 2. without top and trainable layers
# MobileNet from scratch -> https://github.com/rcmalli/keras-mobilenet/blob/master/keras_mobilenet/mobilenet.py
# -- 1. as is plus fiddle with activation ,learning rate, alpha(width), etc

#### Test Model Parameters

In [ ]:
# Import the SGD optimizer
from keras.optimizers import SGD

lr = 0.001
alpha = 2
model_history = []

# Loop over learning rates
for alpha in alpha_to_test:
    print('\n\nTesting model with alpha parameter: %f\n'%alpha )
    
    # Build new model to test, unaffected by previous models
    model = nn_mobilenet.model(alpha=alpha, input_shape=input_shape)
    
    # Create SGD optimizer with specified learning rate: my_optimizer
    # Use SGD rather than Adam to help narrow down best learning rate to start with
    my_optimizer = SGD(lr=lr)
    
    # Compile the model
    model.compile(optimizer=my_optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    
    # Fit the model
    model_history.append(
                            model.fit_generator(
                                    train_batches_aug, 
                                    steps_per_epoch=steps_per_epoch,
                                    validation_data=val_batches_aug,
                                    validation_steps=val_steps,
                                    epochs=30,
                                    verbose=0)
                        )

**Run Time**

In [ ]:
import nn_mobilenet

In [ ]:
# Learning rate at 0.001 with alpha 2, using SGD

# Alpha / width
alpha = 1

# Create model
model = nn_mobilenet.model(alpha=alpha, input_shape=input_shape, shallow=False)

# Optimizer
optimizer = SGD(lr=0.00001)

# Compile the model
model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

# Checkpoint callback
outputFolder = model_path+'nn_mobilenet/'
if not os.path.exists(outputFolder):
    os.makedirs(outputFolder)
filepath=outputFolder+"weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1,
                             save_best_only=True, save_weights_only=True)

# Darly stopping callback
earlystop = EarlyStopping(monitor='val_loss', patience=5,verbose=1, mode='auto')

callbacks_list = [checkpoint, earlystop]


# Fit the model
start = time.time()
print("Model started training at",time.ctime())
history =  model.fit_generator(train_batches_aug,
                               steps_per_epoch=steps_per_epoch,
                               validation_data=val_batches_aug,
                               validation_steps=val_steps,
                               epochs=50,
                               verbose=0,
                               callbacks=callbacks_list)
end = time.time()
print("Model took %0.2f seconds to train"%(end - start))
# plot model history
helper_keras.plot_model_history(history)

In [ ]:
helper_keras.plot_model_history(history) #SGD: lr = 0.0.00001, swallow=True